In [17]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import os
from datetime import datetime

In [18]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    torch.cuda.set_device(device)
else:
    print('there is no Cuda')
    device = torch.device('cpu')
    torch.cuda.set_device(device)

In [19]:
today = datetime.today()
Data_file = os.path.join('../Data',str(today.date()),'Data.csv')
log_dir = os.path.join('D:/TV/Save_data',str(today.date()),'log')
pth_dir = os.path.join("D:/TV/Save_data',str(today.date()),'pth")

In [27]:
class CustomDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]


data = pd.read_csv(Data_file,header=0,skiprows=[0,2,3])
data_cleaned = data.loc[:, (data.max(axis=0) != data.min(axis=0)) & ~data.isna().all(axis=0)]
FyFL = data_cleaned.pop('Car.FyFL')

data_cleaned = data_cleaned.values
FyFL = FyFL.values
data_cleaned = torch.tensor(data_cleaned, dtype=torch.float32).to(device)
FyFL = torch.tensor(FyFL, dtype=torch.float32).to(device)


In [28]:
data_cleaned.type()

'torch.cuda.FloatTensor'

In [ ]:
def save_checkpoint(model, filename):
    torch.save(model, filename)

In [16]:

X_train, X_val, y_train, y_val = train_test_split(data_cleaned, FyFL, test_size=0.2, random_state=42)

train_dataset = CustomDataset(X_train,y_train)
val_dataset = CustomDataset(X_val,y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)


In [ ]:
class DNN(nn.Module):
    def __init__(self,input_size,hidden_size,depth = 2):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.depth = depth
        self.fc1 = nn.Linear(self.input_size,self.hidden_size)
        self.fc2 = nn.Linear(self.hidden_size,self.hidden_size)
        self.fc3 = nn.Linear(self.hidden_size,1)
        self.relu = nn.ReLU()
    

    def forward(self,x):
        x = self.relu(self.fc1(x))
        for i in range(self.depth):
            x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))

        return x


In [ ]:
model = DNN()
model.to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr= 0.001)
loss_log = []


In [ ]:
# Training loop
for epoch in tqdm(range(10)):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        train_loss = criterion(outputs, labels)
        train_loss.backward()
        optimizer.step()

    # Validation loop
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for inputs, labels in val_loader:
            outputs = model(inputs)
            val_loss += criterion(outputs, labels).item()
        val_loss /= len(val_loader)
    loss_log.append([train_loss,val_loss])

    print(f'Epoch {epoch+1}, Loss: {train_loss.item():.4f}, Validation Loss: {val_loss:.4f}')
    save_checkpoint(epoch, model, os.path.join(pth_dir, f'model_epoch_{epoch}.pth'))

loss_log = pd.DataFrame(loss_log, columns=['Train loss', 'Validation Loss'])
with pd.ExcelWriter(os.path.join(log_dir,'log'+'.xlsx'), engine="xlsxwriter") as writer:
            loss_log.to_excel(writer, sheet_name="variables")